In [ ]:
#%matplotlib notebook
#%matplotlib inline
# Importing necessary packages:
from glob import glob
import pandas as pd
import seaborn as sns
from PipeLine import *

In [ ]:
log_pairs = glob('../cylinder_logs_csvs/*.csv')
log_pairs = PipeLine.file_reader(log_pairs,extensions=['_runtime.csv','_details.csv'])

In [ ]:
runtime_dfs = []
details_dfs = []
for log_pair in log_pairs:
    runtime_df = pd.read_csv(log_pair[0])
    runtime_df.wall_time = pd.to_timedelta(runtime_df.wall_time)
    runtime_df['total_time_hr'] = runtime_df.wall_time.dt.total_seconds() / 3600
    runtime_dfs.append(runtime_df)
    
    details_df = pd.read_csv(log_pair[1])
    details_dfs.append(details_df)
    
    
df_of_runtimes = pd.concat(runtime_dfs)
df_of_details = pd.concat(details_dfs)

In [ ]:
df_of_details.loc[df_of_details.dangerous!=0,:]

In [ ]:
data = df_of_runtimes
facet_grid = sns.lmplot(x="natoms", y="total_time_hr", hue='ncores', data=data,legend_out=False)#,height=4, aspect=16/9);
facet_grid.tight_layout()
facet_grid.set_axis_labels("Particles","Wall time (hours)")
facet_grid.legend.set_title("CPUs")
facet_grid.set(xlim=(0, 140000), ylim=(0, 45))
#plt.legend(title="CPUs", loc='upper right')
output="../walltime_vs_natoms.pdf"
facet_grid.savefig(output)

g = sns.FacetGrid(data, col="ncores",sharex=False,sharey=False)
g.map(sns.regplot, "natoms", "total_time_hr")
g.set_titles(col_template="{col_name} CPUs")
g.set_axis_labels("Particles" ,"Wall time (hours)")
#g.axes[0][0].set_xlim((1000, 5000))
#g.axes[0][1].set_xlim((5000, 25000))
#g.axes[0][2].set_xlim((25000, 50000))
#g.axes[0][3].set_xlim((50000, 100000))
#g.axes[0][4].set_xlim((100000,150000))
#g.axes[0][0].set_xlim((1000, 5000))
g.tight_layout()
output="../walltime_vs_natoms_separate.pdf"
g.savefig(output)